In [3]:
import numpy as np

class perceptron(object):
    """
    Parameters
    -----------
    eta: float
        Learning rate (between 0.0 and 1.0)
    n_iter: int
        Number of iteration that model runs for
    random_state: int
        Random state the model uses, used to seed the model
    Attributes
    ----------
    w_ : 1d-array
        Weights of the model
    errors_ : list
        Number of missclassifications (updates) in each epoch
    """
    def __init__(self,eta=0.01,n_iter=50,random_state=1):
        self.eta = eta
        self.n_iter = 50
        self.random_state = random_state
        
    def fit(self,X,y):
        """Fit training data
        Parameters
        ----------
        X : {array-like}, shape = [n_samples,n_features]
        Training vector where n_sample is number of samples and n_features: number of features
        y: {array-like},shape = [n_samples]
        Returns
        -------
        self:object
        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0,scale =0.01, size = 1+X.shape[1])
        self.errors_ = []
        for _ in range(self.n_iter):
            errors = 0
            for xi,target in zip(X,y):
                update = self.eta*(target - self.predict(xi))
                self.w_[1:] += update*xi
                self.w_[0] += update
                errors = int(update!=0)
            self.errors_.append(errors)
        return self
    
    def net_input(self,X):
        """Calcualte net input"""
        return np.dot(X,self.w_[1:]) + w_[0]
    def predict(self,X):
        """Return class label after each unit step"""
        return np.where(self.net_input(X) >= 0,1,-1)

In [ ]:
Test = perceptron()